## Métrica desconocimiento

In [1]:
from dbconn import querydb, querydbtopandas 
import pandas as pd 

In [2]:
#preguntar a que se refiere con id_tipo en encuestas
#preguntar xq tenemos prestador, efector, demandante 

In [17]:

df_enc = querydbtopandas("""select top 10 * from  DBA.ft_prest_encuestas  """)

In [19]:
df_enc.columns

Index(['id_encuesta', 'id_tie_mes_e', 'fec_encuesta', 'id_tie_mes_c',
       'fec_consumo', 'id_afiliado', 'id_medio', 'id_tipo', 'id_moti_desconoc',
       'id_transac', 'id_transac_tipo', 'id_transac_canal',
       'id_exp_expediente', 'id_ofi', 'id_rece', 'id_farma', 'id_produ',
       'id_prestador', 'id_emp_empresa', 'id_prest_efector',
       'id_prest_demandante', 'id_prest_prestacion', 'i_valor', 'i_dif_dias',
       'i_CNT_resp', 'i_CNT_desc', 'i_CNT', 'flg_origen'],
      dtype='object')

In [3]:
mayor = "2019-12"
menor = "2019-10"

In [44]:
df = querydbtopandas(f"""select    a11.id_encuesta  id_encuesta,
                
                a13.id_pre_tipo,


                a11.id_afiliado  id_afi_afiliado,

                a11.id_prestador  id_pre_prestador,

                max(a13.desc_pre_nombre)  desc_pre_nombre,

                max(a13.vigente)  vigente,

                a11.id_transac_tipo  id_transac_tipo,

                a11.id_prest_demandante  id_prest_demandante,

                max(a16.desc_pre_nombre)  desc_pre_nombre_demandante,

                a11.id_prest_prestacion  id_pres_prestacion,

                max(a18.desc_pres_prestacion)  desc_pres_prestacion,

                max(a18.prestac_pres_prestacion)  prestac_pres_prestacion,

                sum(distinct a11.i_CNT_desc)  QDESCONOCIDAS

from      DBA.ft_prest_encuestas              a11

                join        DBA.d_pre_prestador   a13

                  on        (a11.id_prestador = a13.id_pre_prestador)

                join        DBA.d_pre_prestador_demandante  a16

                  on        (a11.id_prest_demandante = a16.id_pre_prestador)

                join        DBA.d_pres_prestacion a18

                  on        (a11.id_prest_prestacion = a18.id_pres_prestacion)


where (cast(a11.fec_encuesta as date) >= date("{menor}") and cast(a11.fec_encuesta as date) < date("{mayor}")

and a11.id_moti_desconoc <> 1)

group by             a11.id_encuesta,

                    a13.id_pre_tipo,

                a11.id_afiliado,

                a11.id_prestador,
                
                a11.id_prest_demandante,
                
                 a16.id_pre_prestador, 

                a13.id_pre_especialidad,

                a11.id_transac_tipo,

                a11.id_prest_efector,

                a11.id_prest_prestacion,

                DATE(a11.fec_consumo),

                a11.id_tie_mes_c""")

In [45]:
df.columns

Index(['id_encuesta', 'id_pre_tipo', 'id_afi_afiliado', 'id_pre_prestador',
       'desc_pre_nombre', 'vigente', 'id_transac_tipo', 'id_prest_demandante',
       'desc_pre_nombre_demandante', 'id_pres_prestacion',
       'desc_pres_prestacion', 'prestac_pres_prestacion', 'QDESCONOCIDAS'],
      dtype='object')

In [46]:
clinicas_propias_id = [1632,1763,2086,26564,55060 ,64677,75470,78363 ,85316,85401 ,99785 ,118209 ,122803,135464 ,182730,187243 ,196559 ,196561 ,206939 ,210252 ,211421, 213729 ,217747 ,226756,235779,240895] 
cma = df[df["id_pre_prestador"].isin(clinicas_propias_id)][["id_pre_prestador", "desc_pre_nombre"]].rename(columns={"desc_pre_nombre": "cma"})
df = pd.merge(df, cma, how="outer").fillna("No aplica")

In [47]:
propios = df.cma.value_counts().index[1:]

In [48]:
df.columns

Index(['id_encuesta', 'id_pre_tipo', 'id_afi_afiliado', 'id_pre_prestador',
       'desc_pre_nombre', 'vigente', 'id_transac_tipo', 'id_prest_demandante',
       'desc_pre_nombre_demandante', 'id_pres_prestacion',
       'desc_pres_prestacion', 'prestac_pres_prestacion', 'QDESCONOCIDAS',
       'cma'],
      dtype='object')

In [51]:
df_part_equipos = df[df["id_pre_tipo"].isin(["P","E"])][["id_encuesta", "id_afi_afiliado", "id_pre_prestador", "desc_pre_nombre", "cma" ]]
df_inst_circ = df[df["id_pre_tipo"].isin(["I","C"])][["id_encuesta","id_afi_afiliado", "id_prest_demandante", "desc_pre_nombre_demandante","cma"]]
df_inst_circ.rename(columns={"id_prest_demandante": "id_pre_prestador", "desc_pre_nombre_demandante":"desc_pre_nombre"},inplace=True)

In [52]:
df = pd.concat([df_part_equipos, df_inst_circ])

In [53]:
df[df.cma.isin(propios)]

,id_encuesta,id_afi_afiliado,id_pre_prestador,desc_pre_nombre,cma
460,568704,2905311,0,Sin Asignar,SWISS MEDICAL CENTER DE LOS ARCOS
461,568704,2905311,0,Sin Asignar,SWISS MEDICAL CENTER DE LOS ARCOS
462,568704,2905311,0,Sin Asignar,SWISS MEDICAL CENTER DE LOS ARCOS
463,568704,2905311,0,Sin Asignar,SWISS MEDICAL CENTER DE LOS ARCOS
464,568704,2905311,0,Sin Asignar,SWISS MEDICAL CENTER DE LOS ARCOS
...,...,...,...,...,...
4522,564822,2821431,243300,"Sarochar,Sofía",SWISS MEDICAL CENTER ALMAGRO
4805,535280,2773419,249069,"Salcedo,Anabel",SWISS MEDICAL CENTER BARRIO PARQUE
4806,535280,2773419,249069,"Salcedo,Anabel",SWISS MEDICAL CENTER BARRIO PARQUE
4807,576321,1642340,67880,"Lynch,Enrique D.",SWISS MEDICAL CENTER BARRIO PARQUE


In [54]:
df = df[df["id_pre_prestador"] != 0]

In [55]:
df.cma.value_counts()

No aplica                                           1191
SWISS MEDICAL CENTER MICROCENTRO SEDE 25 DE MAYO     340
SWISS MEDICAL CENTER BARRIO NORTE                    330
SWISS MEDICAL CENTER MICROCENTRO SEDE SARMIENTO       25
SWISS MEDICAL CENTER BERAZATEGUI                       9
SWISS MEDICAL CENTER OLIVOS                            4
SWISS MEDICAL CENTER BARRIO PARQUE                     4
SWISS MEDICAL CENTER ALMAGRO                           1
Name: cma, dtype: int64

In [56]:
df.cma.unique()

array(['No aplica', 'SWISS MEDICAL CENTER OLIVOS',
       'SWISS MEDICAL CENTER MICROCENTRO SEDE 25 DE MAYO',
       'SWISS MEDICAL CENTER BARRIO NORTE',
       'SWISS MEDICAL CENTER MICROCENTRO SEDE SARMIENTO',
       'SWISS MEDICAL CENTER BERAZATEGUI', 'SWISS MEDICAL CENTER ALMAGRO',
       'SWISS MEDICAL CENTER BARRIO PARQUE'], dtype=object)

In [57]:
#count de encuestas
encuestas_desconocidas = df.groupby(["id_pre_prestador","desc_pre_nombre","cma"])["id_encuesta"].nunique()

In [58]:
#count de afiliados
afiliados_desconocidos = df.groupby(["id_pre_prestador","desc_pre_nombre", "cma"])["id_afi_afiliado"].nunique()

In [59]:
df = pd.concat([encuestas_desconocidas, afiliados_desconocidos],axis=1).reset_index()

In [60]:
df.rename(columns = {"id_pre_prestador":"Prestador",
                     "desc_pre_nombre":"Razón Social",
                     "id_encuesta":"Q desconocidas",
                     "id_afi_afiliado":"Q afiliados"},inplace=True)

In [61]:
final_1 = df

In [63]:
oct_nov_desconociento_condemandantes = df.head(100)

In [64]:
oct_nov_desconociento_condemandantes.to_excel("oct_nov_desconociento_condemandantes.xlsx")

In [65]:
oct_nov_desconociento_condemandantes

,Prestador,Razón Social,cma,Q desconocidas,Q afiliados
0,105,"Cohen,Diego E.",No aplica,10,3
1,172,"Pasqualini,Rodolfo S.",No aplica,2,2
2,232,"Fioroni,Andrea P.",No aplica,1,1
3,292,"Tejss de Acri,Cristina",No aplica,1,1
4,334,"Barclay,Fernando E.",No aplica,1,1
...,...,...,...,...,...
95,31000,"Gomez,Leonardo J.",No aplica,1,1
96,31077,"Barcia,Ricardo E.",No aplica,1,1
97,31340,"Franco,Fabio G.",No aplica,5,2
98,31463,"Antebi de Sporn,Noemí E.",No aplica,1,1


In [216]:
final_1

,Prestador,Razón Social,cma,Q desconocidas,Q afiliados
0,105,"Cohen,Diego E.",No aplica,9,2
1,172,"Pasqualini,Rodolfo S.",No aplica,1,1
2,432,"Juarez Goñi,Francisco",No aplica,1,1
3,529,"Schvartzman,Javier A.",No aplica,1,1
4,636,"Cahué,Jorge",No aplica,2,1
...,...,...,...,...,...
211,215973,"Davin,Florencia",No aplica,1,1
212,219030,"Tagliabue,Marcela",No aplica,1,1
213,219373,"Striebeck,Miranda E.",No aplica,1,1
214,238913,"Rueda,Adriana S.",No aplica,2,1
